In [208]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re

In [209]:
####  FUNCIONES  ####

# Limpieza de texto
def limpiar_texto(texto):
    # Eliminar valores numéricos utilizando expresiones regulares
    texto_sin_numeros = re.sub(r'\d+', '', str(texto))

    # Remover tildes y caracteres especiales de entonacion
    texto_sin_tildes = unidecode(texto_sin_numeros)
    #texto_sin_tildes = unidecode(texto)

    # Quitar caracteres no alfabéticos y no espacios en blanco
    texto_limpio = re.sub(r'[^a-zA-Z\s]', '', texto_sin_tildes)

    # Aplicar trim para eliminar espacios en blanco al principio y al final
    texto_limpio = texto_limpio.strip()
    
    return texto_limpio


# Funcion like ampliada
def asignar_ciudad(row):
    # el diccionario se usa para buscar una coincidencia parcial entre las claves y los valores en la columna
    condiciones = {
        'BOGO': 'BOGOTA',
        'GOTA': 'BOGOTA',
        'LOCALID': 'BOGOTA',
        'MEDEL': 'MEDELLIN',
        'COMUNA': 'MEDELLIN',
        'CALIFORNIA': 'SACRAMENTO',
        'MEXICALI': 'MEXICALI',
        'CALI': 'CALI',
        'DUPAR': 'VALLEDUPAR',
        'VALLE': 'CALI',
        'CARTAGE': 'CARTAGENA',
        'QUILLA': 'BARRANQUILLA',
        'BARRANQ': 'BARRANQUILLA',
        'VILLAVI': 'VILLAVICENCIO',
        'YOPAL': 'YOPAL',
        'TUNJA': 'TUNJA',
        'CUCUT': 'CUCUTA',
        'MANIZA': 'MANIZALES',
        'IBAG': 'IBAGUE',
        'NEIV': 'NEIVA', 
        'MARTA': 'SANTA MARTA',
        'BUCARA': 'BUCARAMANGA',
        'MONTERI': 'MONTERIA',
        'POPAYA': 'POPAYAN',
        'ARMENI': 'ARMENIA',
        'PEREIR': 'PEREIRA',
        'UBATE': 'UBATE',
        'CHIA': 'CHIA',
        'SOLEDA': 'SOLEDAD',
        'BELLO': 'BELLO',
        'SOACH': 'SOACHA',
        'BUGA': 'BUGA',
        'PAST': 'PASTO',
        'MANGA': 'BUCARAMANGA',
        # Puedes agregar más condiciones aquí según sea necesario
    }
    
    for condicion, ciudad in condiciones.items():
        if condicion in row:
            return ciudad
    
    return row


# Elimina la palabra contenida en la lista de la cadena
# def eliminar_departamento(row):
#     reg = ['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'BOGOTA' 'BOLIVAR' 'BOYACA'
#             'CALDAS' 'CAQUETA' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCO' 'CUNDINAMARCA'
#             'CORDOBA' 'GUAINIA' 'GUAVIARE' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META'
#             'NARINO' 'NORTE DE SANTANDER' 'PUTUMAYO' 'QUINDIO' 'RISARALDA'
#             'SAN ANDRES' 'SANTANDER' 'SUCRE' 'TOLIMA'
#             'VALLE DEL CAUCA' 'VAUPES' 'VICHADA']
#     for i in reg:
#         if i in row:
#             row = row.replace(i, '').strip()
    
#     return row


# Elimina la palabra que 
def eliminar_departamento(row):
    regiones = ['AMAZONAS','ANTIOQUIA', 'ARAUCA','ATLANTICO','BOLIVAR','BOYACA','CALDAS','CAQUETA',
                'CASANARE','CAUCA','CESAR','CHOCO','CUNDINAMARCA','CORDOBA','GUAINIA','GUAVIARE','HUILA',
                'LA GUAJIRA','MAGDALENA','META','NARINO','NORTE DE SANTANDER','PUTUMAYO','QUINDIO',
                'RISARALDA','ISLA','ISLAS','SANTANDER','SUCRE','TOLIMA','VALLE DEL CAUCA','VAUPES','VICHADA']

    # Crear un diccionario donde las claves sean los valores de la lista y los valores sean None
    dic= {region: '' for region in regiones}
    
    for condicion, ciudad in dic.items():
        if condicion in row:
            row = row.replace(condicion, ciudad).strip()
    
    return row



# Reemplaza algunas versiones adicionales de ciudad
def replace_cities_aliases(ciudad):
    # bogota_aliases = ['BOOGOTA','BTA','BOGTA','BOG','BOOGTA','BGTA','BGOTA','BGA','CUNDINAMARCA','BGOOTA','BOOTA',
    #                   'BIGOTA','BOGITA','CEDRITOS','BOBOTA','B OGOTA','BOGATA','BOHOTA','BOGPTA',]
    bogota_aliases = ['BOTOTA','BTA','BOGTA','BOG','BOOGTA','BGTA','KENEDY','BGA','CUNDINAMARCA','BGOOTA','BOOTA',
                      'BOFOTA','BOGITA','CEDRITOS','BOBOTA','NOGOTA','BOGATA','BOHOTA','BOGPTA','CIUDAD VERDE','CIUDADVERDE',
                      'TINTAL','HAYUELOS','SALITRE']
    barranquilla_aliases = ['ATLANTICO','BAQ','BQLLLA','BQ','BQLLA','BARRAQNUILLA','BARRANUILLA','BQUILLLA']
    cali_aliases = ['VALLE DEL CAUCA']
    cartagena_aliases = ['CTG']
    ibague_aliases = ['IABGUE']
    medellin_aliases = ['ANTIOQUIA','POBLADO','MED','MDELLIN','RIO NEGRO']
    if ciudad in bogota_aliases:
        return 'BOGOTA'
    elif ciudad in barranquilla_aliases:
        return 'BARRANQUILLA'
    elif ciudad in cali_aliases:
        return 'CALI'
    elif ciudad in cartagena_aliases:
        return 'CARTAGENA'
    elif ciudad in medellin_aliases:
        return 'MEDELLIN'
    elif ciudad in ibague_aliases:
        return 'IBAGUE'
    else:
        return ciudad
    

# Reemplaza algunas versiones adicionales de ciudad
def ultimos_casos(ciudad):
    santanderdequilichao = ['DE QUILICHAO']
    barranquilla_aliases = ['BAQ','BQLLLA','BQ','BQLLA']
    if ciudad in santanderdequilichao:
        return 'SANTANDER DE QUILICHAO'
    elif ciudad in barranquilla_aliases:
        return 'BARRANQUILLA'
    else:
        return ciudad

In [210]:
# regiones = ['AMAZONAS','ANTIOQUIA', 'ARAUCA','ATLANTICO','BOLIVAR','BOYACA','CALDAS','CAQUETA',
#                 'CASANARE','CAUCA','CESAR','CHOCO','CUNDINAMARCA','CORDOBA','GUAINIA','GUAVIARE','HUILA',
#                 'LA GUAJIRA','MAGDALENA','META','NARINO','NORTE DE SANTANDER','PUTUMAYO','QUINDIO',
#                 'RISARALDA','SAN ANDRES','SANTANDER','SUCRE','TOLIMA','VALLE DEL CAUCA','VAUPES','VICHADA']

# # Crear un diccionario donde las claves sean los valores de la lista y los valores sean None
# dic= {region: '' for region in regiones}

# print(dic.items())
# print(type(dic.items()))

#eliminar_departamento('ABEJORRAL ANTIOQUIA')

In [211]:
## IMPORTE DE TODAS LAS TABLAS REQUERIDAS

inpath = "E:/OneDrive/Datasets/Tuboleta/"
#inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/"

#A) TABLA dirty_cities
cities = pd.read_excel(inpath + 'dirty_cities.xlsx').\
    sort_values('MAIN_ADDR_TOWN', ascending = True)

#cities = cities['MAIN_ADDR_TOWN']

#################
# Descriptivos
# print(cities.head(5))
# print(' ')
# print('Dimensiones:')
# print(cities.shape)

print('----------------------------------------------------------')
#B) TABLA de codigos postales
codpost = pd.read_csv(inpath + 'codpost.csv', sep = '|',
                        # usecols=['%Cliente','Fecha', 'Producto'],
                        dtype={'Codigo Postal': 'object'})

#################
# Descriptivos
# print(codpost.head(5))
# print(' ')
# print('Dimensiones:')
# print(codpost.shape)


print('----------------------------------------------------------')
#C) TABLA de municipios de Colombia
dim_towns = pd.read_csv(inpath + 'dim_municipios.csv', sep = '|',
                        # usecols=['%Cliente','Fecha', 'Producto'],
                        dtype={'Codigo Postal': 'object'})

#################
# Descriptivos
# print(dim_towns.head(5))
# print(' ')
# print('Dimensiones:')
# print(dim_towns.shape)


print('----------------------------------------------------------')
#D) TABLA localidades de Bogota
loc_bog = pd.read_excel(inpath + 'localidades_bogota.xlsx')

#################
# Descriptivos
# print(loc_bog.head(5))
# print(' ')
# print('Dimensiones:')
# print(loc_bog.shape)

----------------------------------------------------------
----------------------------------------------------------
----------------------------------------------------------


In [212]:
# left join
cities_ = pd.merge(cities, codpost[['Codigo Postal', 'Ciudad']],\
                          left_on='MAIN_ADDR_TOWN', right_on= codpost['Codigo Postal'].astype(str),\
                          how='left')

# Rellenar los valores nulos en 'Ciudad' con los valores de 'MAIN_ADDR_TOWN'
cities_['Ciudad'].fillna(cities_['MAIN_ADDR_TOWN'], inplace=True)

#################
# Descriptivos
# print(cities_.head(5))
# print(' ')
# print('Dimensiones:')
# print(cities_.shape)
# print('Summary:')
# codpost.info()
# print(' ')
# print('Descriptivos:')
# print(cities_.describe().round(2).transpose())
# print(' ')
# print('Nulos por campo:')
# print(cities_.isnull().sum())  # total de nulls por variable

In [213]:
# Valores con codigo postal matched
prueba = cities_[cities_['Codigo Postal'].notnull()]

print(prueba['count'].sum())

#################
# Descriptivos
print(prueba.head(5))
print(' ')
print('Dimensiones:')
print(prueba.shape)

6415
     MAIN_ADDR_TOWN  count Codigo Postal            Ciudad
1698         110111    901        110111  BOGOTA   USAQUEN
1720         110121    137        110121  BOGOTA   USAQUEN
1727         110131     61        110131  BOGOTA   USAQUEN
1730         110141     40        110141  BOGOTA   USAQUEN
1735         110151     14        110151  BOGOTA   USAQUEN
 
Dimensiones:
(495, 4)


In [214]:
# def asignar_ciudad(row):
#     patrones = {
#         r'^POPAYA$': 'POPAYAN',
#         r'^ARMENI$': 'ARMENIA',
#         r'^PEREIR$': 'PEREIRA',
#         r'^UBATE$': 'UBATE',
#         # Puedes agregar más patrones aquí según sea necesario
#     }
    
#     for patron, ciudad in patrones.items():
#         if re.match(patron, row['Ciudad']):
#             return ciudad
    
#     return row['Ciudad']

In [215]:
total = sum(cities_['count'])
print('num registros: ', total)

dim_municipios_ = cities_.sort_values(by='count', ascending=False).\
       assign(Ciudad = lambda x: x['Ciudad'].apply(limpiar_texto),
              PARTICIPACION = lambda x: x['count']/total,
              TOTAL = lambda x: np.cumsum(x['PARTICIPACION'])).\
       rename(columns = {'count': 'COUNT'})
       #groupby('DEPARTAMENTO').agg({'COUNT': 'sum','PORCENTAJE': 'sum'}).reset_index().\

num registros:  3126459


In [216]:
# left join con localidades
dim_municipios_1 = pd.merge(dim_municipios_, loc_bog,\
                          left_on='Ciudad', right_on= 'LOCALIDAD',\
                          how='left')

print('Dimensiones:')
print(dim_municipios_1.shape)

#dim_municipios_1 = dim_municipios_1[dim_municipios_1['CIUDAD'].notnull()]

Dimensiones:
(33185, 8)


In [217]:
# Valores con localidad matched
prueba = dim_municipios_1[dim_municipios_1['CIUDAD'].notnull()]

print(prueba['COUNT'].sum())

#################
# Descriptivos
print(prueba.head(5))
print(' ')
print('Dimensiones:')
print(prueba.shape)

6088
    MAIN_ADDR_TOWN  COUNT Codigo Postal    Ciudad  PARTICIPACION     TOTAL  \
113           SUBA   1050           NaN      SUBA       0.000336  0.925503   
166       ENGATIVA    512           NaN  ENGATIVA       0.000164  0.937861   
181        KENNEDY    458           NaN   KENNEDY       0.000146  0.940213   
195       FONTIBON    426           NaN  FONTIBON       0.000136  0.942195   
196           BOSA    426           NaN      BOSA       0.000136  0.942331   

    LOCALIDAD  CIUDAD  
113      SUBA  BOGOTA  
166  ENGATIVA  BOGOTA  
181   KENNEDY  BOGOTA  
195  FONTIBON  BOGOTA  
196      BOSA  BOGOTA  
 
Dimensiones:
(87, 8)


In [218]:
dim_municipios_1 = dim_municipios_1.assign(CIUDAD = lambda x: x['CIUDAD'].fillna(dim_municipios_1['Ciudad'])).\
                                    assign(CIUDAD = lambda x: x['CIUDAD'].apply(asignar_ciudad)).\
                                    assign(CIUDAD = lambda x: x['CIUDAD'].apply(replace_cities_aliases))
                                    #assign(CIUDAD = lambda x: x['CIUDAD'].apply(eliminar_departamento)).\
#################
# Descriptivos
print(dim_municipios_1.head(5))
print(' ')
print('Dimensiones:')
print(dim_municipios_1.shape)

  MAIN_ADDR_TOWN    COUNT Codigo Postal        Ciudad  PARTICIPACION  \
0            NaN  1034485           NaN           nan       0.330881   
1         BOGOTA   615709           NaN        BOGOTA       0.196935   
2    BOGOTÁ D.C.   173332           NaN     BOGOTA DC       0.055440   
3   BARRANQUILLA    92630           NaN  BARRANQUILLA       0.029628   
4       MEDELLIN    86092           NaN      MEDELLIN       0.027537   

      TOTAL LOCALIDAD        CIUDAD  
0  0.330881       NaN           nan  
1  0.527816       NaN        BOGOTA  
2  0.583256       NaN        BOGOTA  
3  0.612884       NaN  BARRANQUILLA  
4  0.640420       NaN      MEDELLIN  
 
Dimensiones:
(33185, 8)


In [219]:

prueba = dim_municipios_1[['CIUDAD','COUNT','PARTICIPACION']].\
    groupby('CIUDAD').agg({'COUNT': 'sum','PARTICIPACION': 'sum'}).\
        sort_values(by='PARTICIPACION', ascending=False).reset_index()

#################
# Descriptivos
#print(prueba.head(5))
print(' ')
print('Dimensiones de nueva base dirty, original 33k:')
print(prueba.shape)


# Escritura
prueba.to_csv(inpath +'prueba.csv', 
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
               sep = '|',
               decimal = '.')  


 
Dimensiones de nueva base dirty, original 33k:
(16659, 3)


In [220]:
# left join para traer la region y el departamento
dim_municipios = pd.merge(dim_municipios_1, dim_towns,\
                          left_on='CIUDAD', right_on= 'MUNICIPIO',\
                          how='left')

dim_municipios['MUNICIPIO'] = np.where(
    (dim_municipios['CIUDAD'] == '') |
    (dim_municipios['CIUDAD'] == 'nan') |
    (dim_municipios['CIUDAD'] == 'None') |
    (dim_municipios['CIUDAD'].isnull()),
    'NO DISPONIBLE',
    dim_municipios['MUNICIPIO']
)

#################
# Descriptivos
print(dim_municipios.head(5))
print(' ')
print('Dimensiones:')
print(dim_municipios.shape)
print('Summary:')
dim_municipios.info()
print(' ')
print('Descriptivos:')
print(dim_municipios.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(dim_municipios.isnull().sum())  # total de nulls por variable

  MAIN_ADDR_TOWN    COUNT Codigo Postal        Ciudad  PARTICIPACION  \
0            NaN  1034485           NaN           nan       0.330881   
1         BOGOTA   615709           NaN        BOGOTA       0.196935   
2    BOGOTÁ D.C.   173332           NaN     BOGOTA DC       0.055440   
3   BARRANQUILLA    92630           NaN  BARRANQUILLA       0.029628   
4       MEDELLIN    86092           NaN      MEDELLIN       0.027537   

      TOTAL LOCALIDAD        CIUDAD                           REGION  \
0  0.330881       NaN           nan                              NaN   
1  0.527816       NaN        BOGOTA            Región Centro Oriente   
2  0.583256       NaN        BOGOTA            Región Centro Oriente   
3  0.612884       NaN  BARRANQUILLA                    Región Caribe   
4  0.640420       NaN      MEDELLIN  Región Eje Cafetero - Antioquia   

  DEPARTAMENTO      MUNICIPIO       CAPITAL      PAIS  
0          NaN  NO DISPONIBLE           NaN       NaN  
1       BOGOTA        

In [221]:
# valida si hay municipios con el mismo nombre
# prueba = dim_towns['MUNICIPIO'].value_counts().reset_index()
# prueba = prueba[prueba['count'] > 1]

# # Escritura
# prueba.to_csv(inpath +'prueba.csv', 
#                index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
#                sep = '|',
#                decimal = '.')  

In [222]:
# Valores con MUNICIPIO LIMPIO
notnull = dim_municipios[dim_municipios['MUNICIPIO'].notnull()]
notnull = notnull[['MAIN_ADDR_TOWN','COUNT','PARTICIPACION','Ciudad','CIUDAD','MUNICIPIO','REGION','DEPARTAMENTO','CAPITAL','PAIS']]

print('Registros: ', notnull['COUNT'].sum())
print('Percent: ', notnull['PARTICIPACION'].sum())

#################
# Descriptivos
print(notnull.head(5))
print(' ')
print('Dimensiones:')
print(notnull.shape)

Registros:  3044578
Percent:  0.9738103074436608
  MAIN_ADDR_TOWN    COUNT  PARTICIPACION        Ciudad        CIUDAD  \
0            NaN  1034485       0.330881           nan           nan   
1         BOGOTA   615709       0.196935        BOGOTA        BOGOTA   
2    BOGOTÁ D.C.   173332       0.055440     BOGOTA DC        BOGOTA   
3   BARRANQUILLA    92630       0.029628  BARRANQUILLA  BARRANQUILLA   
4       MEDELLIN    86092       0.027537      MEDELLIN      MEDELLIN   

       MUNICIPIO                           REGION DEPARTAMENTO       CAPITAL  \
0  NO DISPONIBLE                              NaN          NaN           NaN   
1         BOGOTA            Región Centro Oriente       BOGOTA        BOGOTA   
2         BOGOTA            Región Centro Oriente       BOGOTA        BOGOTA   
3   BARRANQUILLA                    Región Caribe    ATLANTICO  BARRANQUILLA   
4       MEDELLIN  Región Eje Cafetero - Antioquia    ANTIOQUIA      MEDELLIN   

       PAIS  
0       NaN  
1  COLOMB

In [223]:
null = dim_municipios[dim_municipios['MUNICIPIO'].isnull()]

print('Registros: ', null['COUNT'].sum())
print('Percent: ', null['PARTICIPACION'].sum())

#################
# Descriptivos
print('Dimensiones:')
print(null.shape)

Registros:  81881
Percent:  0.026189692556339298
Dimensiones:
(19993, 13)


In [224]:
# Aplicamos la funcion de eliminar departamento para limpiar valores estilo: 'ABEJORRAL ANTIOQUIA'
null_ = null[['MAIN_ADDR_TOWN','COUNT','PARTICIPACION','Ciudad','CIUDAD']].\
    assign(CIUDAD = lambda x: x['CIUDAD'].apply(eliminar_departamento))



# left join para traer la region y el departamento NUEVAMENTE
null_ = pd.merge(null_, dim_towns,\
                          left_on='CIUDAD', right_on= 'MUNICIPIO',\
                          how='left')

null2 = null_[null_['MUNICIPIO'].notnull()]

print('Registros: ', null2['COUNT'].sum())
print('Percent: ', null2['PARTICIPACION'].sum())

#################
# Descriptivos
print('Dimensiones:')
print(null2.shape)

Registros:  9680
Percent:  0.003096154467402259
Dimensiones:
(2220, 10)


In [225]:
prueba = null_[null_['MUNICIPIO'].isnull()]

print('Registros: ', prueba['COUNT'].sum())
print('Percent: ', prueba['PARTICIPACION'].sum())

#################
# Descriptivos
print(prueba.head(50))
print(' ')
print('Dimensiones:')
print(prueba.shape)

# Escritura
prueba.to_csv(inpath +'prueba.csv', 
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
               sep = '|',
               decimal = '.')  

Registros:  72201
Percent:  0.02309353808893704
          MAIN_ADDR_TOWN  COUNT  PARTICIPACION                Ciudad  \
0                 CIUDAD   3803       0.001216                CIUDAD   
1                  QUITO   2326       0.000744                 QUITO   
2          -SELECCIONAR-   2044       0.000654           SELECCIONAR   
3                 PANAMA   1975       0.000632                PANAMA   
4                   LIMA   1965       0.000629                  LIMA   
5              GUAYAQUIL   1626       0.000520             GUAYAQUIL   
6                CIUDAD    1522       0.000487                CIUDAD   
7                  MIAMI   1182       0.000378                 MIAMI   
8                      I    604       0.000193                     I   
9       CIUDAD DE MEXICO    499       0.000160      CIUDAD DE MEXICO   
11  CEDULA DE CIUDADANIA    446       0.000143  CEDULA DE CIUDADANIA   
13              NEW YORK    408       0.000130              NEW YORK   
14              

In [226]:
# UNIMOS AMBAS TABLAS PARA TENER LA LIMPIEZA COMPLETA
dim_municipios_ = pd.concat([notnull, null_])

#################
# Descriptivos
print('Dimensiones:')
print(dim_municipios_.shape)

Dimensiones:
(33185, 10)


: 

In [188]:
#dim_municipios_ = dim_municipios[['MAIN_ADDR_TOWN','COUNT','MUNICIPIO','REGION','DEPARTAMENTO','CAPITAL','PAIS']]


# Escritura
dim_municipios_.to_excel(inpath +'cleaned_cities.xlsx',
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
               #sep = '|',
               #decimal = '.'
               )